In [1]:
import sys
import ROOT as RT
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, DelphesSchema
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
import pprint

Welcome to JupyROOT 6.24/06


Helper function for streamlining the import of the root files

In [2]:
def getEvents(fileURL, treename):
    return NanoEventsFactory.from_root(fileURL, treepath = treename, schemaclass = DelphesSchema).events()


Import root files and save them to dict called datasets

In [3]:
filedir   = "root://cms-xrd-global.cern.ch//store/user/christiw/BDecay_delphes/v2/"
filenames = {'1e1mm': "m1_ct10mm.root",
             '1e2mm': "m1_ct100mm.root",
             '1e3mm': "m1_ct1000mm.root",
             '1e4mm': "m1_ct10000mm.root"}
treename  = "Delphes"

# TODO: ak.zip this into one awkward array
datasets = {}
for key in filenames:
    datasets[key] = getEvents(filedir + filenames[key], treename)


In [12]:
datasets['1e4mm'].fields

['MissingET.offsets',
 'MissingET',
 'CscCluster130.offsets',
 'CscCluster130',
 'CscClusterID.offsets',
 'CscClusterID',
 'Particle.offsets',
 'Particle',
 'Cscllp.offsets',
 'Cscllp',
 'Vertex.offsets',
 'Vertex',
 'Event.offsets',
 'Event',
 'Electron.offsets',
 'Electron',
 'llp.offsets',
 'llp',
 'higgs.offsets',
 'higgs',
 'Jet.offsets',
 'Jet',
 'Muon.offsets',
 'Muon',
 'Weight.offsets',
 'Weight',
 'MuonIso.offsets',
 'MuonIso']

A precut that takes in event.llp or event.CscCluster130 and cuts everything outside the bounding box.

In [4]:
#takes in either event.llp or event.CscCluster130
def inCSC(llp):
    
    r = np.sqrt(llp.X**2 + llp.Y**2) 
    
    trueIn = ((abs(llp.Eta) < 2.4) &
               (abs(llp.Z) > 4000) & (abs(llp.Z) < 11000) &
               (r < 6955)) 
    
    return llp[trueIn]

Generates histograms for efficiency mapping of llp and CscCluster130 for a given variable, inside of the detector

In [5]:
# initialize some dicitonaries to store histograms in an organized manner
pos_histos = {}
Ehad_histos  = {}

# for variable binning
Ehad_bins = np.array([0,25,50,75,100,150,200,250])*1.

# initialize dictionaries
for ctau in datasets:
    pos_histos[ctau] = {'reco': RT.TH1F('Z decay pos (mm)', ctau + ' reco', 25, 6500, 11000),
                        'real': RT.TH1F('Z decay pos (mm)', ctau + ' real', 25, 6500, 11000)}
    
    Ehad_histos[ctau]  = {'reco': RT.TH1F('Hadronic Energy (GeV)', ctau + ' reco', 7, Ehad_bins),
                          'real': RT.TH1F('Hadronic Energy (GeV)', ctau + ' real', 7, Ehad_bins)}   
                         # real means generated llp (events.llp)
                         # reco means reconstructed (events.CscCluster130)

pprint.pprint(pos_histos)

{'1e1mm': {'real': <cppyy.gbl.TH1F object at 0x564362325eb0>,
           'reco': <cppyy.gbl.TH1F object at 0x5643623274a0>},
 '1e2mm': {'real': <cppyy.gbl.TH1F object at 0x56436232e850>,
           'reco': <cppyy.gbl.TH1F object at 0x5643623486a0>},
 '1e3mm': {'real': <cppyy.gbl.TH1F object at 0x56436232e2d0>,
           'reco': <cppyy.gbl.TH1F object at 0x5643623295b0>},
 '1e4mm': {'real': <cppyy.gbl.TH1F object at 0x5643622f9770>,
           'reco': <cppyy.gbl.TH1F object at 0x564362326d40>}}


Warning in <TROOT::Append>: Replacing existing TH1: Z decay pos (mm) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Hadronic Energy (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Z decay pos (mm) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Z decay pos (mm) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Hadronic Energy (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Hadronic Energy (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Z decay pos (mm) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Z decay pos (mm) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Hadronic Energy (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Hadronic Energy (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replaci

In [6]:
# loops over each ctau in datasets
for ctau in datasets:
    #sys.stdout.write("\r Reading " + filenames[ctau])
    
    # cut's out everything not in muon cap
    realLLP = inCSC(datasets[ctau].llp)
    recoLLP = inCSC(datasets[ctau].CscCluster130)
    
    # flattens to have an iterable list of all the Z positions (within the endcap)
    realLLP_zpos = ak.flatten(realLLP.Z, axis = None)
    recoLLP_zpos = ak.flatten(recoLLP.Z, axis = None)
    
    # flattens to have an iterable list of all the hadronic energies (of particles within the endcap)
    realLLP_Ehad = ak.flatten(realLLP.Ehad, axis = None)
    recoLLP_Ehad = ak.flatten(recoLLP.Ehad, axis = None)
    
    # histogram maker. Split into different loops for readability
    for z in realLLP_zpos:
        pos_histos[ctau]['real'].Fill(z)
        
    for z in recoLLP_zpos:
        pos_histos[ctau]['reco'].Fill(z)
        
    for Ehad in realLLP_Ehad:
        Ehad_histos[ctau]['real'].Fill(Ehad)
        
    for Ehad in recoLLP_Ehad:
        Ehad_histos[ctau]['reco'].Fill(Ehad)
        

In [7]:
# calculates and adds efficiencies to the histogram dictionaries
for ctau in datasets:
    pos_histos[ctau]['ratio'] = RT.TEfficiency(pos_histos[ctau]['reco'], pos_histos[ctau]['real'])
    pos_histos[ctau]['ratio'].SetMarkerStyle(20)
    
    Ehad_histos[ctau]['ratio'] = RT.TEfficiency(Ehad_histos[ctau]['reco'], Ehad_histos[ctau]['real'])
    Ehad_histos[ctau]['ratio'].SetMarkerStyle(20)
    
pprint.pprint(pos_histos)

{'1e1mm': {'ratio': <cppyy.gbl.TEfficiency object at 0x564362ac04a0>,
           'real': <cppyy.gbl.TH1F object at 0x564362325eb0>,
           'reco': <cppyy.gbl.TH1F object at 0x5643623274a0>},
 '1e2mm': {'ratio': <cppyy.gbl.TEfficiency object at 0x56435da74800>,
           'real': <cppyy.gbl.TH1F object at 0x56436232e850>,
           'reco': <cppyy.gbl.TH1F object at 0x5643623486a0>},
 '1e3mm': {'ratio': <cppyy.gbl.TEfficiency object at 0x564362adc5e0>,
           'real': <cppyy.gbl.TH1F object at 0x56436232e2d0>,
           'reco': <cppyy.gbl.TH1F object at 0x5643623295b0>},
 '1e4mm': {'ratio': <cppyy.gbl.TEfficiency object at 0x564362a99f50>,
           'real': <cppyy.gbl.TH1F object at 0x5643622f9770>,
           'reco': <cppyy.gbl.TH1F object at 0x564362326d40>}}


Saves efficiency histograms to pdf with proper labels

In [8]:
EhadFileName = 'EhadEff.pdf'
c = RT.TCanvas("canvas")
c.Print(EhadFileName + "[")
    
for ctau in Ehad_histos:
    te = Ehad_histos[ctau]['ratio']
    te.SetTitle(ctau+"; Hadronic Energy ; \epsilon")
    te.Draw()
    c.Print(EhadFileName)
    c.Clear()
    
c.Print(EhadFileName + "]") #closes pdf
c.Close() #closes canvas

Info in <TCanvas::Print>: pdf file EhadEff.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file EhadEff.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file EhadEff.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file EhadEff.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file EhadEff.pdf
Info in <TCanvas::Print>: pdf file EhadEff.pdf has been closed


In [9]:
PosFileName = 'PosEff.pdf'
c = RT.TCanvas("canvas")
c.Print(PosFileName + "[")
    
for ctau in pos_histos:
    te = pos_histos[ctau]['ratio']
    te.SetTitle(ctau+"; Z decay Pos ; \epsilon")
    te.Draw()
    c.Print(PosFileName)
    c.Clear()
    
c.Print(PosFileName + "]") #closes pdf
c.Close() #closes canvas

Info in <TCanvas::Print>: pdf file PosEff.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file PosEff.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file PosEff.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file PosEff.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file PosEff.pdf
Info in <TCanvas::Print>: pdf file PosEff.pdf has been closed
